<a href="https://colab.research.google.com/github/iam-fern/Generate_data/blob/main/ex_sampling_c_1000rep_n20_x2_sas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

import statistics as stats
import scipy.stats as ss
import scipy.stats as st
from scipy.stats import f_oneway
from scipy import stats
import scipy.stats as stats
import time
import statsmodels.api as sm

from statsmodels.formula.api import ols

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv('https://github.com/iam-fern/Research/raw/main/sampling_1000rep_n20_x2_sas.csv', index_col=['rep', 'index'])
df

x1         x2  ...       y_hat        e_hat
rep   index                        ...                         
0.0   0.0    33.744364  36.270505  ...  386.767734   516.173266
      1.0    34.324181  25.244346  ...  411.567395   699.958370
      2.0    41.742944  24.773295  ...  506.419415   432.979532
      3.0    35.133322  44.163120  ...  397.979751   483.873347
      4.0    30.580127  49.630104  ...  356.389752 -1653.951491
...                ...        ...  ...         ...          ...
999.0 15.0   27.932361  21.434511  ...  319.949593  1465.247922
      16.0   11.948298  29.146464  ...  216.056068    39.206884
      17.0   40.017216  18.459214  ...  440.098453 -2766.523356
      18.0   21.815521  22.083059  ...  250.074365   105.331387
      19.0   18.318184  29.506951  ...  216.041887   434.410177

[20000 rows x 7 columns]

In [3]:
# ลบข้อมูลที่มี missing values
table = df.dropna()
table

x1         x2  ...       y_hat        e_hat
rep   index                        ...                         
0.0   0.0    33.744364  36.270505  ...  386.767734   516.173266
      1.0    34.324181  25.244346  ...  411.567395   699.958370
      2.0    41.742944  24.773295  ...  506.419415   432.979532
      3.0    35.133322  44.163120  ...  397.979751   483.873347
      4.0    30.580127  49.630104  ...  356.389752 -1653.951491
...                ...        ...  ...         ...          ...
999.0 15.0   27.932361  21.434511  ...  319.949593  1465.247922
      16.0   11.948298  29.146464  ...  216.056068    39.206884
      17.0   40.017216  18.459214  ...  440.098453 -2766.523356
      18.0   21.815521  22.083059  ...  250.074365   105.331387
      19.0   18.318184  29.506951  ...  216.041887   434.410177

[20000 rows x 7 columns]

In [4]:
table.reset_index(inplace=True)
table

,rep,index,x1,x2,x3,e,Y,y_hat,e_hat
0,0.0,0.0,33.744364,36.270505,19.062328,505.164523,902.941000,386.767734,516.173266
1,0.0,1.0,34.324181,25.244346,27.656113,710.383500,1111.525765,411.567395,699.958370
2,0.0,2.0,41.742944,24.773295,36.048022,456.148187,939.398947,506.419415,432.979532
3,0.0,3.0,35.133322,44.163120,18.046456,463.310299,881.853098,397.979751,483.873347
4,0.0,4.0,30.580127,49.630104,22.125915,-1680.119027,-1297.561739,356.389752,-1653.951491
...,...,...,...,...,...,...,...,...,...
19995,999.0,15.0,27.932361,21.434511,14.813268,1464.626124,1785.197516,319.949593,1465.247922
19996,999.0,16.0,11.948298,29.146464,49.941691,51.691814,255.262952,216.056068,39.206884
19997,999.0,17.0,40.017216,18.459214,10.294129,-2760.350408,-2326.424903,440.098453,-2766.523356
19998,999.0,18.0,21.815521,22.083059,12.275425,97.892058,355.405752,250.074365,105.331387


In [5]:
table.set_index(['rep', 'index'], inplace=True)

In [6]:
table

x1         x2  ...       y_hat        e_hat
rep   index                        ...                         
0.0   0.0    33.744364  36.270505  ...  386.767734   516.173266
      1.0    34.324181  25.244346  ...  411.567395   699.958370
      2.0    41.742944  24.773295  ...  506.419415   432.979532
      3.0    35.133322  44.163120  ...  397.979751   483.873347
      4.0    30.580127  49.630104  ...  356.389752 -1653.951491
...                ...        ...  ...         ...          ...
999.0 15.0   27.932361  21.434511  ...  319.949593  1465.247922
      16.0   11.948298  29.146464  ...  216.056068    39.206884
      17.0   40.017216  18.459214  ...  440.098453 -2766.523356
      18.0   21.815521  22.083059  ...  250.074365   105.331387
      19.0   18.318184  29.506951  ...  216.041887   434.410177

[20000 rows x 7 columns]

In [7]:
def get_result(run_replicate):
    new_table = table.loc[run_replicate]

    # show index
    new_table.reset_index(inplace=True)

    new_table.drop(['index'], axis=1, inplace=True)


    # แบ่งข้อมูลออกเป็น 2 กลุ่มด้วย k-mean

    #Set number of clusters
    x = new_table[["x1", "x2", "x3"]]
    k_means = KMeans(n_clusters=2)

    #Run the clustering algorithm
    model = k_means.fit(x)
    model

    #Generate cluster predictions and store in y_kmeans
    y_kmeans = k_means.predict(x)
    y_kmeans

    # kmean_clusters.astype(float)
    kmean_clusters = pd.DataFrame({"clusters" : y_kmeans})

    # เชื่อมข้อมูลทั้ง 2 DataFrame เข้าด้วยกันด้วยคำสั่ง pd.concat
    data_clusters = pd.concat([new_table, kmean_clusters], axis=1)

    data_sort_clusters = data_clusters.sort_values('clusters')

    # print(data_clusters.groupby('clusters').groups)

    data_sort_clusters.reset_index(inplace=True)

    # กลุ่มที่ 1
    data_cluster1 = data_sort_clusters[data_sort_clusters.clusters == 1]

    y = data_cluster1[['Y']]
    x = data_cluster1[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model1 = sm.OLS(y, x).fit()

    DF1 = model1.df_resid

    SSR1 = model1.ssr

    MSE1 = model1.mse_resid

    # กลุ่มที่ 2
    data_cluster0 = data_sort_clusters[data_sort_clusters.clusters == 0]

    y = data_cluster0[['Y']]
    x = data_cluster0[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model0 = sm.OLS(y, x).fit()

    DF0 = model0.df_resid

    SSR0 = model0.ssr

    MSE0 = model0.mse_resid

    df1 = DF1+1
    df0 = DF0+1

    # คำนวณค่าสถิติ
    GQ_test_clusters = (SSR1/df1) / (SSR0/df0)


    # คำนวณค่า p-value
    # sf(x, dfn, dfd, loc=0, scale=1) : Survival function (also defined as 1 - cdf, but sf is sometimes more accurate)
    two_tailed = 2.0*(1.0-(ss.f.cdf(GQ_test_clusters, df1, df0)))

    # pdf(x, dfn, dfd, loc=0, scale=1) : Probability density function
    p_greater = ss.f.cdf(GQ_test_clusters, df1, df0)
    
    return df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater

# สร้าง list ของ replicates
list_replicate=list({i[0] for i in table.index})
list_replicate.sort()

# สร้าง list เพื่อเก็บค่า ทุกรอบที่ run
list_df0=[]
list_df1=[]
list_SSR0=[]
list_SSR1=[]
list_MSE0=[]
list_MSE1=[]
list_GQ_test_clusters=[]
list_result_test=[]
list_two_tailed=[]
list_p_greater=[]
list_result_test01=[]

# ใช้ for loop สำหรับการ run ทุก replicates
for run_replicate in list_replicate:
    df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater=get_result(run_replicate)

    list_df0.append(df0)
    list_df1.append(df1)
    list_SSR0.append(SSR0)
    list_SSR1.append(SSR1)
    list_MSE0.append(MSE0)
    list_MSE1.append(MSE1)
    list_GQ_test_clusters.append(GQ_test_clusters)
    list_two_tailed.append(two_tailed)
    list_p_greater.append(p_greater)

/usr/local/lib/python3.7/dist-packages/statsmodels/regression/linear_model.py:1665: RuntimeWarning: divide by zero encountered in double_scalars
  return self.ssr/self.df_resid
/usr/local/lib/python3.7/dist-packages/statsmodels/regression/linear_model.py:1665: RuntimeWarning: divide by zero encountered in double_scalars
  return self.ssr/self.df_resid
/usr/local/lib/python3.7/dist-packages/statsmodels/regression/linear_model.py:1665: RuntimeWarning: divide by zero encountered in double_scalars
  return self.ssr/self.df_resid
/usr/local/lib/python3.7/dist-packages/statsmodels/regression/linear_model.py:1665: RuntimeWarning: divide by zero encountered in double_scalars
  return self.ssr/self.df_resid


In [8]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1, 
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)
two_tailed_new=two_tailed_new[(two_tailed_new["two_tailed"]<=1) & (two_tailed_new["greater"]<=1)]
two_tailed_new["Result_test"]=two_tailed_new["two_tailed"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                  if p_value < alpha else 'Reject005=1 : Homoscedasticity')
#two_tailed_new_show=two_tailed_new.drop(columns="greater")
two_tailed_new.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed,greater,Result_test
Replicate,,,,,,,,,,
0.0,8.0,6.0,3635271.0595230325,10493304.136319734,519324.43707471894,2098660.8272639466,3.8487012254858795,0.08315831215382352,0.9584208439230882,Reject005=1 : Homoscedasticity
3.0,6.0,8.0,1029302.3834960234,9866932.092238445,205860.47669920468,1409561.727462635,7.189528740858515,0.027078675527314022,0.986460662236343,Reject005=0 : Heteroscedasticity
4.0,8.0,6.0,5589936.550737591,7935746.700240505,798562.3643910845,1587149.340048101,1.8928650628287564,0.3961334068277109,0.8019332965861445,Reject005=1 : Homoscedasticity
7.0,8.0,6.0,1789775.1196834422,16929490.51180922,255682.15995477745,3385898.102361844,12.612005702558909,0.002138789500707672,0.9989306052496462,Reject005=0 : Heteroscedasticity
8.0,8.0,6.0,1018075.0104373866,2651506.826854194,145439.28720534095,530301.3653708388,3.4725755956692566,0.1079593408496835,0.9460203295751582,Reject005=1 : Homoscedasticity
9.0,5.0,9.0,1244551.3978894474,2707592.037548147,311137.84947236185,338449.0046935184,1.2086425688716136,0.878477441603347,0.5607612791983265,Reject005=1 : Homoscedasticity
12.0,9.0,5.0,9608973.793637581,10761226.958955137,1201121.7242046976,2690306.7397387843,2.0158457023730154,0.3403261008703431,0.8298369495648285,Reject005=1 : Homoscedasticity
18.0,9.0,5.0,3217108.474337535,4824373.432849098,402138.5592921919,1206093.3582122745,2.6992786374468003,0.18554838368397486,0.9072258081580126,Reject005=1 : Homoscedasticity
21.0,7.0,7.0,3418953.7771327826,9453298.355320059,569825.6295221305,1575549.7258866765,2.764968166152812,0.203056192171017,0.8984719039144915,Reject005=1 : Homoscedasticity


In [9]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1,
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)

two_tailed_new["two_tailed_new"]=two_tailed_new[["two_tailed","greater"]].apply(lambda value: value[0] if value[0]<0.5 else 2*value[1], axis=1)
two_tailed_new["Result_test"]=two_tailed_new["two_tailed_new"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                      if p_value < alpha else 'Reject005=1 : Homoscedasticity')


two_tailed_new_show=two_tailed_new.drop(columns=["two_tailed", "greater"])
two_tailed_new_show.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed_new,Result_test
Replicate,,,,,,,,,
0.0,8.0,6.0,3635271.0595230325,10493304.136319734,519324.43707471894,2098660.8272639466,3.8487012254858795,0.08315831215382352,Reject005=1 : Homoscedasticity
1.0,8.0,6.0,12768358.696787464,694799.7362228174,1824051.2423982092,138959.9472445635,0.07255432513265572,0.004885494039304535,Reject005=0 : Heteroscedasticity
2.0,6.0,8.0,5122705.72531019,3483985.3421254368,1024541.145062038,497712.1917322053,0.510079857541662,0.3718023378681557,Reject005=1 : Homoscedasticity
3.0,6.0,8.0,1029302.3834960234,9866932.092238445,205860.47669920468,1409561.727462635,7.189528740858515,0.027078675527314022,Reject005=0 : Heteroscedasticity
4.0,8.0,6.0,5589936.550737591,7935746.700240505,798562.3643910845,1587149.340048101,1.8928650628287564,0.3961334068277109,Reject005=1 : Homoscedasticity
...,...,...,...,...,...,...,...,...,...
995.0,5.0,9.0,290044.49955569865,15016833.590481715,72511.12488892466,1877104.1988102144,28.76346678121821,0.001755517565075193,Reject005=0 : Heteroscedasticity
996.0,6.0,8.0,9531732.458596433,6002091.11807656,1906346.4917192864,857441.5882966515,0.4722717887971738,0.3218416039143774,Reject005=1 : Homoscedasticity
997.0,7.0,7.0,10196658.661405563,271958.7343891125,1699443.1102342606,45326.45573151875,0.026671358081101465,0.00010003349446299667,Reject005=0 : Heteroscedasticity


In [10]:
two_tailed_new_show['Result_test'].value_counts()


Reject005=1 : Homoscedasticity      553
Reject005=0 : Heteroscedasticity    447
Name: Result_test, dtype: int64